# Week-18 Homework Assignment

#### 1.	What is a neural network? What are the general steps required to build a neural network? 

#### Neural network
A large connected network which is passing the inputs(values) through hidden layers and there is some computation happening
to get the desired output. Purely considering the Neural Network has three layers: Input Layer, Hidden Layer, 
and Output Layer. While the inputs and outputs correspond to visible things 
and they can be stored as data, the values in the hidden layer aren't something we have 
data about, or anything we observe directly. Each node in the hidden layer is an aggregation of information that comes from input 
data, and each node adds to the model's ability to capture interactions.
So the more nodes we have, the more interactions we can capture.

#### The general steps required to build a neural network:

When the hidden layers recieve information from inputs

The connections between the two layers assign weights to each input randomly

Then bias will be added to every input after weights are multiplied with them individually

The weighted sum is then transferred to the activation function(allows the model to capture non-linearities)

combination = bias +weights * inputs

An activation function is something applied to the value coming into a node, which then transforms 
it into the value stored in that node, or the node output

Weights are adjusted, and the output is back-propagated to minimize error based on loss obtained in previous epoch.


#### 2.	Generally, how do you check the performance of a neural network? Why? 

Generally as a performance metric we use Mean-Squared-Error, Root-Mean-Squared-Error, Mean_Absolute_Error, 
R square for regression type of problems and when it comes to classification problems we use True Positive rate,
False Positive rate, accuracy, precision nad recall.

Its hard to make accurate prediction with multiple points. So, we use a loss function to
aggregate all the errors into a single measure of the model's predictive performance. A common loss 
function for regression tasks is mean-squared error. Here we square each error, 
and take the average of that as a measure of model quality.

Lower values mean a better model, so our goal is to find the weights giving the lowest value for the loss function. 
We do this with an algorithm called gradient descent.

#### 3.	Create a neural network using keras to predict the outcome of either of these datasets: 

Cardiac Arrhythmia: https://archive.ics.uci.edu/ml/datasets/Arrhythmia 

Abalone age: https://archive.ics.uci.edu/ml/datasets/Abalone

The common name for Abalone is sea snail. we can determine the age of the abalone by cutting the shell through the cone,
stained and the rings are counted using the microscope. All these is a time consuming process that can be simplified 
using neural networks. Here, we can predict their age using physical measurements like length, height, weight and
other features.

Lets import our Abalone dataset and some dependent libraries

In [17]:
import pandas as pd 
import numpy as np

In [18]:
# import the dataset (Abalone dataset)
target_url = (r"A:\launch_code_STL\Final_Homework\week-17\abalone.data")

In [19]:
abalone_df = pd.read_csv(target_url,header=None, prefix="V")
abalone_df.columns = ['Sex', 'Length', 'Diameter', 'Height',
                   'Whole weight', 'Shucked weight',
                   'Viscera weight', 'Shell weight', 'Rings']

print(abalone_df.shape)
print(abalone_df.head())

(4177, 9)
  Sex  Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0   M   0.455     0.365   0.095        0.5140          0.2245          0.1010   
1   M   0.350     0.265   0.090        0.2255          0.0995          0.0485   
2   F   0.530     0.420   0.135        0.6770          0.2565          0.1415   
3   M   0.440     0.365   0.125        0.5160          0.2155          0.1140   
4   I   0.330     0.255   0.080        0.2050          0.0895          0.0395   

   Shell weight  Rings  
0         0.150     15  
1         0.070      7  
2         0.210      9  
3         0.155     10  
4         0.055      7  


The dataset contains 4177 samples and each sample has 9 features('Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 
'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings') The Rings feature is the target variable and as given 
in the documentation, adding 1.5 to the ring attribute gives the age of the abalone.

###  Preprocessing

#### Encoding the input columns

Out of the 8 input columns, 'Sex' column is a categorical attribute, we need to perform one hot encoding, we use 
get_dummies() function.

In [20]:
# One hot encoding the sex attribute.
df_dummies = pd.get_dummies(abalone_df['Sex'], prefix = "Sex_") # drop_first parameter is set to True, this means
                                                                           # that if there are n categories in the column,
                                                                           # n-1 columns are returned instead of n.

# Inserting dummy columns
for column in df_dummies.columns:
    abalone_df[column] = df_dummies[column]  # one-hot encoded columns are appended to the data frame
    
# Dropping the original column
abalone_df = abalone_df.drop(columns = ['Sex'])     # original 'Sex' attribute column is dropped

In [21]:
print(abalone_df)

      Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0      0.455     0.365   0.095        0.5140          0.2245          0.1010   
1      0.350     0.265   0.090        0.2255          0.0995          0.0485   
2      0.530     0.420   0.135        0.6770          0.2565          0.1415   
3      0.440     0.365   0.125        0.5160          0.2155          0.1140   
4      0.330     0.255   0.080        0.2050          0.0895          0.0395   
...      ...       ...     ...           ...             ...             ...   
4172   0.565     0.450   0.165        0.8870          0.3700          0.2390   
4173   0.590     0.440   0.135        0.9660          0.4390          0.2145   
4174   0.600     0.475   0.205        1.1760          0.5255          0.2875   
4175   0.625     0.485   0.150        1.0945          0.5310          0.2610   
4176   0.710     0.555   0.195        1.9485          0.9455          0.3765   

      Shell weight  Rings  Sex__F  Sex_

#### Encoding the output columns

The attribute 'Rings' is with continuous values ranging from 1 to 29. we can split these into 3 categories — 
less than 10(young), between 10 and 20 (middle age), between 20 and 30(old).


In [22]:
def rings_label(x):
    if x<=10:
        return 'young'
    if x<=20:
        return 'middle age'
    if x<=30:
        return 'old'
    
abalone_df['Rings'] = abalone_df['Rings'].apply(rings_label)

In [23]:
print(abalone_df)

      Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0      0.455     0.365   0.095        0.5140          0.2245          0.1010   
1      0.350     0.265   0.090        0.2255          0.0995          0.0485   
2      0.530     0.420   0.135        0.6770          0.2565          0.1415   
3      0.440     0.365   0.125        0.5160          0.2155          0.1140   
4      0.330     0.255   0.080        0.2050          0.0895          0.0395   
...      ...       ...     ...           ...             ...             ...   
4172   0.565     0.450   0.165        0.8870          0.3700          0.2390   
4173   0.590     0.440   0.135        0.9660          0.4390          0.2145   
4174   0.600     0.475   0.205        1.1760          0.5255          0.2875   
4175   0.625     0.485   0.150        1.0945          0.5310          0.2610   
4176   0.710     0.555   0.195        1.9485          0.9455          0.3765   

      Shell weight       Rings  Sex__F 

In [24]:
# One hot encoding the Rings attribute.
df_dummies = pd.get_dummies(abalone_df['Rings'])

# Inserting dummy columns
for column in df_dummies.columns:
    abalone_df[column] = df_dummies[column]
    
# Dropping the original column
df = abalone_df.drop(columns = ['Rings'])

In [25]:
# defining the input and output columns to separate the dataset in the later cells.

input_columns = df.columns.tolist()
input_columns.remove('young')
input_columns.remove('middle age')
input_columns.remove('old')

output_columns = ['young', 'middle age', 'old']

In [26]:
print(df)

      Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0      0.455     0.365   0.095        0.5140          0.2245          0.1010   
1      0.350     0.265   0.090        0.2255          0.0995          0.0485   
2      0.530     0.420   0.135        0.6770          0.2565          0.1415   
3      0.440     0.365   0.125        0.5160          0.2155          0.1140   
4      0.330     0.255   0.080        0.2050          0.0895          0.0395   
...      ...       ...     ...           ...             ...             ...   
4172   0.565     0.450   0.165        0.8870          0.3700          0.2390   
4173   0.590     0.440   0.135        0.9660          0.4390          0.2145   
4174   0.600     0.475   0.205        1.1760          0.5255          0.2875   
4175   0.625     0.485   0.150        1.0945          0.5310          0.2610   
4176   0.710     0.555   0.195        1.9485          0.9455          0.3765   

      Shell weight  Sex__F  Sex__I  Sex

In [27]:
# Splitting into train, val and test set -- 80-10-10 split
from sklearn.model_selection import train_test_split

# First, an 80-20 split
train_df, val_test_df = train_test_split(df, test_size = 0.2)

# Then split the 20% into half
val_df, test_df = train_test_split(val_test_df, test_size = 0.5)

# Splitting into X (input) and y (output)

X_train, y_train = np.array(train_df[input_columns]), np.array(train_df[output_columns])

X_val, y_val = np.array(val_df[input_columns]), np.array(val_df[output_columns])

X_test, y_test = np.array(test_df[input_columns]), np.array(test_df[output_columns])

In [28]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

Xtrain = sc.fit_transform(X_train)
Xval = sc.transform(X_val)
Xtest = sc.transform(X_test)

In [31]:
# Import necessary modules
import keras
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras import layers
import keras.callbacks
from keras.callbacks import EarlyStopping


model = Sequential([
    layers.Dense(32, activation = 'relu', input_shape = X_train[0].shape),
    layers.Dense(8, activation = 'relu'),
    layers.Dense(3, activation = 'softmax')])

early_stopping_monitor = EarlyStopping(patience=2)

model.compile(optimizer = 'Adam', loss = 'mean_absolute_error', metrics = ['mean_absolute_error'])

# model.compile(optimizer = 'Adam', loss = 'CategoricalCrossentropy', metrics = ['accuracy'])

history = model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = 10, callbacks = [early_stopping_monitor])

Epoch 1/10
105/105 [==============================] - 0s 2ms/step - loss: 0.3225 - mean_absolute_error: 0.3225 - val_loss: 0.2564 - val_mean_absolute_error: 0.2564
Epoch 2/10
105/105 [==============================] - 0s 1ms/step - loss: 0.2370 - mean_absolute_error: 0.2370 - val_loss: 0.2365 - val_mean_absolute_error: 0.2365
Epoch 3/10
105/105 [==============================] - 0s 1ms/step - loss: 0.2308 - mean_absolute_error: 0.2308 - val_loss: 0.2351 - val_mean_absolute_error: 0.2351
Epoch 4/10
105/105 [==============================] - 0s 1ms/step - loss: 0.2302 - mean_absolute_error: 0.2302 - val_loss: 0.2347 - val_mean_absolute_error: 0.2347
Epoch 5/10
105/105 [==============================] - 0s 1ms/step - loss: 0.2300 - mean_absolute_error: 0.2300 - val_loss: 0.2346 - val_mean_absolute_error: 0.2346
Epoch 6/10
105/105 [==============================] - 0s 1ms/step - loss: 0.2300 - mean_absolute_error: 0.2300 - val_loss: 0.2346 - val_mean_absolute_error: 0.2346
Epoch 7/10
105/1

In [32]:
model.evaluate(X_test, y_test)

14/14 [==============================] - 0s 847us/step - loss: 0.2361 - mean_absolute_error: 0.2361


[0.23607148230075836, 0.23607148230075836]

In [33]:
# Model summary for number of parameters used in the algorithm
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 32)                352       
                                                                 
 dense_12 (Dense)            (None, 8)                 264       
                                                                 
 dense_13 (Dense)            (None, 3)                 27        
                                                                 
Total params: 643
Trainable params: 643
Non-trainable params: 0
_________________________________________________________________


#### 4.	Write another algorithm to predict the same result as the previous question using either KNN or logistic regression.

In [36]:
abalone_df = pd.read_csv(target_url,header=None, prefix="V")
abalone_df.columns = ['Sex', 'Length', 'Diameter', 'Height',
                   'Whole weight', 'Shucked weight',
                   'Viscera weight', 'Shell weight', 'Rings']
# abalone_df = pd.DataFrame(abalone_df, columns=abalone_df.columns)
# y = abalone_df.Rings

print(abalone_df.shape)
print(abalone_df.head())

(4177, 9)
  Sex  Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0   M   0.455     0.365   0.095        0.5140          0.2245          0.1010   
1   M   0.350     0.265   0.090        0.2255          0.0995          0.0485   
2   F   0.530     0.420   0.135        0.6770          0.2565          0.1415   
3   M   0.440     0.365   0.125        0.5160          0.2155          0.1140   
4   I   0.330     0.255   0.080        0.2050          0.0895          0.0395   

   Shell weight  Rings  
0         0.150     15  
1         0.070      7  
2         0.210      9  
3         0.155     10  
4         0.055      7  


In [37]:
sex_dictionary = {'M': 1, 'F' : 2,'I':3}
abalone_df['Sex'] = abalone_df['Sex'].apply(lambda x: sex_dictionary[x])

In [38]:
abalone_df.dtypes

Sex                 int64
Length            float64
Diameter          float64
Height            float64
Whole weight      float64
Shucked weight    float64
Viscera weight    float64
Shell weight      float64
Rings               int64
dtype: object

In [39]:
abalone_df.select_dtypes(include=['object']).isnull().sum()
abalone_df.select_dtypes(include=[np.number]).isnull().sum()

Sex               0
Length            0
Diameter          0
Height            0
Whole weight      0
Shucked weight    0
Viscera weight    0
Shell weight      0
Rings             0
dtype: int64

In [40]:
# Adding age column in the dataframe for classification. Age is classified based on the number of rings.
#Rings 1-8 -->Age 1, denoting young
# Rings 9-10 -->Age 2, denoting middle
# Rings 11-29 -->Age 3, denoting old

In [41]:
def label_race (row):
    if row['Rings']>=1 and row['Rings']<=8:
        return 1
    elif row['Rings'] >=9 and row['Rings']<=10:
        return 2
    elif row['Rings']>=11 and row['Rings']<=29:
        return 3
    return 0

In [42]:
abalone_df['Age'] = abalone_df.apply(lambda _: '', axis=1)
abalone_df['Age']=abalone_df.apply (lambda row: label_race (row),axis=1)
# abalone_df.Sex = abalone_df.Sex.astype(float)
abalone_df

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,Age
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15,3
1,1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7,1
2,2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9,2
3,1,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10,2
4,3,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7,1
...,...,...,...,...,...,...,...,...,...,...
4172,2,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11,3
4173,1,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10,2
4174,1,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9,2
4175,2,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10,2


In [43]:
abalone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4177 non-null   int64  
 1   Length          4177 non-null   float64
 2   Diameter        4177 non-null   float64
 3   Height          4177 non-null   float64
 4   Whole weight    4177 non-null   float64
 5   Shucked weight  4177 non-null   float64
 6   Viscera weight  4177 non-null   float64
 7   Shell weight    4177 non-null   float64
 8   Rings           4177 non-null   int64  
 9   Age             4177 non-null   int64  
dtypes: float64(7), int64(3)
memory usage: 326.5 KB


In [44]:
# Response variable is age

In [45]:
y = abalone_df[['Age']]
X = abalone_df.drop(['Age','Rings'], axis=1)

In [46]:
# Dividing the dataset into 75% training and 25% test data using train_test_split

# from sklearn import cross_validation
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_validate as cv

# X_train, y_train, X_test, y_test =train_test_split(X, y)
train_X, test_X, train_y, test_y = train_test_split(X, y) # splits 75%/25% by default

In [47]:
print("train_df_x:")
print(train_X.shape)

print("train_df_y:")
print(train_y.shape)

print("test_df_x:")
print(test_X.shape)

print("test_df_y:")
print(test_y.shape)

train_df_x:
(3132, 8)
train_df_y:
(3132, 1)
test_df_x:
(1045, 8)
test_df_y:
(1045, 1)


In [48]:
def getAccuracy(testSet, predictions):

    correct = 0

    for x in range(len(testSet)):

        if testSet[x][-1] == predictions[x]:

            correct += 1

    return (correct/float(len(testSet))) * 100.0

In [49]:
from sklearn.linear_model import LogisticRegression
import math
from sklearn import metrics
from sklearn.metrics import accuracy_score, mean_absolute_error, median_absolute_error, mean_squared_error


#logistic regression
log_reg = LogisticRegression(multi_class='multinomial',solver ='newton-cg')
log_reg.fit(train_X, train_y)
print ("Intercept is ",log_reg.intercept_)
print("Coefficient is ",log_reg.coef_)
y_pred=log_reg.predict(test_X) 

# Accuracy Score
print("Accuracy of Logistic Regression is:", accuracy_score(test_y, y_pred))

#Mean Absolute Error 
mae=mean_absolute_error(test_y,y_pred);
print("MAE:"+str(mae))

#RMSE 
rmse = math.sqrt(mean_squared_error(test_y,y_pred))
print("RMSE:"+str(rmse))

#Median Absolute error
Medae=median_absolute_error(test_y,y_pred)
print("Median Absolute Error:"+str(Medae)) 

#Classification Report
print("Classification report for Test data %s:\n%s\n\n"
     % (log_reg, metrics.classification_report(test_y, y_pred)))

Intercept is  [ 2.42021245 -1.10506645 -1.315146  ]
Coefficient is  [[ 0.28096135 -1.48875905 -1.93005255 -1.4027539  -2.33373752  4.66152265
  -0.84796968 -4.69964869]
 [-0.11703691  2.01882702  1.18868548  0.10818049 -0.78536182  1.42518176
   1.25198571 -0.37094476]
 [-0.16392444 -0.53006797  0.74136707  1.2945734   3.11909933 -6.08670441
  -0.40401603  5.07059345]]
Accuracy of Logistic Regression is: 0.6564593301435406
MAE:0.38181818181818183
RMSE:0.6770326474710262
Median Absolute Error:0.0
Classification report for Test data LogisticRegression(multi_class='multinomial', solver='newton-cg'):
              precision    recall  f1-score   support

           1       0.73      0.80      0.76       359
           2       0.52      0.44      0.48       332
           3       0.68      0.71      0.69       354

    accuracy                           0.66      1045
   macro avg       0.64      0.65      0.65      1045
weighted avg       0.65      0.66      0.65      1045





#### 5.	Create a neural network using pytorch to predict the same result as question 3. 

In [50]:
abalone_df = pd.read_csv(target_url,header=None, prefix="V")
abalone_df.columns = ['Sex', 'Length', 'Diameter', 'Height',
                   'Whole weight', 'Shucked weight',
                   'Viscera weight', 'Shell weight', 'Rings']
abalone_df = pd.DataFrame(abalone_df, columns=abalone_df.columns)
y = abalone_df.Rings

print(abalone_df.shape)
print(abalone_df.head())

(4177, 9)
  Sex  Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0   M   0.455     0.365   0.095        0.5140          0.2245          0.1010   
1   M   0.350     0.265   0.090        0.2255          0.0995          0.0485   
2   F   0.530     0.420   0.135        0.6770          0.2565          0.1415   
3   M   0.440     0.365   0.125        0.5160          0.2155          0.1140   
4   I   0.330     0.255   0.080        0.2050          0.0895          0.0395   

   Shell weight  Rings  
0         0.150     15  
1         0.070      7  
2         0.210      9  
3         0.155     10  
4         0.055      7  


In [51]:
# As Rings: +1.5 gives the age in years , So we will replace rings with age
abalone_df['Age'] = abalone_df['Rings'] + 1.5
abalone_df.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,Age
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,16.5
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,8.5
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,10.5
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,11.5
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,8.5


In [52]:
# One hot encoding the sex attribute.
df_dummies = pd.get_dummies(abalone_df['Sex'], prefix = "Sex_") 

# Inserting dummy columns
for column in df_dummies.columns:
    abalone_df[column] = df_dummies[column]  # one-hot encoded columns are appended to the data frame
    
# Dropping the original column
abalone_df = abalone_df.drop(columns = ['Sex'])   

In [53]:
print(abalone_df)

      Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0      0.455     0.365   0.095        0.5140          0.2245          0.1010   
1      0.350     0.265   0.090        0.2255          0.0995          0.0485   
2      0.530     0.420   0.135        0.6770          0.2565          0.1415   
3      0.440     0.365   0.125        0.5160          0.2155          0.1140   
4      0.330     0.255   0.080        0.2050          0.0895          0.0395   
...      ...       ...     ...           ...             ...             ...   
4172   0.565     0.450   0.165        0.8870          0.3700          0.2390   
4173   0.590     0.440   0.135        0.9660          0.4390          0.2145   
4174   0.600     0.475   0.205        1.1760          0.5255          0.2875   
4175   0.625     0.485   0.150        1.0945          0.5310          0.2610   
4176   0.710     0.555   0.195        1.9485          0.9455          0.3765   

      Shell weight  Rings   Age  Sex__F

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = abalone_df.drop(['Rings','Age'], axis = 1)
y = abalone_df.drop(['Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight',
              'Shell weight', 'Age','Sex__F','Sex__I','Sex__M'], axis = 1)

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

# #Standardize the data
sc= StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [55]:
print("train_df_x:")
print(X_train.shape)

print("train_df_y:")
print(y_train.shape)

print("test_df_x:")
print(X_test.shape)

print("test_df_y:")
print(y_test.shape)

train_df_x:
(2923, 10)
train_df_y:
(2923, 1)
test_df_x:
(1254, 10)
test_df_y:
(1254, 1)


In [56]:
# Import Dependencies
import torch.nn as nn
import torch
import torch.nn.functional as F # Activation function

# Create tensors 
X_train = torch.Tensor(X_train) 
X_test = torch.Tensor(X_test)

y_train = torch.Tensor(y_train)
y_test = torch.Tensor(y_test)

# test_tensor = torch.Tensor(test.values)
print(X_train)

ValueError: could not determine the shape of object type 'DataFrame'

In [ ]:
#artificial neural network
class ANN_Model(nn.Module):
    def __init__(self, input_features=10,hidden1=100,hidden2=100,out_features=2):
        super().__init__()
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
#         self.layer_3_connection = nn.Linear(hidden2, hidden3)
        self.out = nn.Linear(hidden2, out_features)
        
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.softmax(self.layer_2_connection(x))
#         x = F.relu(self.layer_3_connection(x))
        x = self.out(x)
        return x

In [578]:
torch.manual_seed(69)

#create instance of model
ann = ANN_Model()

In [579]:
#loss function
loss_function = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.Adam(ann.parameters(), lr=0.01)

In [580]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = ann.forward(X_train)
    loss = loss_function(y_train, y_pred)
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss}')
        
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() 
    optimizer.step() #perform one optimization step each epoch

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [2]:
# #predictions
# y_pred = []

# with torch.no_grad():
#     for i, data in enumerate(X_test):
#         prediction = ann(data)
#         y_pred.append(prediction.argmax()) 

In [1]:
# # Import mean_squared_error as MSE
# from sklearn.metrics import mean_squared_error as MSE

# # Evaluate the test set RMSE
# rmse_test = MSE(y_test, y_pred)**(1/2)

In [ ]:
######################################################################################################################

In [ ]:
 from collections import OrderedDict

seq_model = nn.Sequential(OrderedDict([
('hidden_linear', nn.Linear(11, 100)),
('hidden_activation', nn.Tanh()),
('output_linear', nn.Linear(100, 7))
]))

def training_loop(n_epochs, optimizer, model, loss_fn, target_train, target_val, data_train, data_val):
    for epoch in range(1, n_epochs+1):
        t_p_train = model(data_train)
        loss_train = loss_fn(t_p_train, target_train)
        
        optimizer.zero_grad() #zero the gradient before running backwards propagation
        loss_train.backward() 
        optimizer.step() #perform one optimization step each epoch
        
        if epoch == 1 or epoch % 1000 ==0:
            print(f'Epoch number: {epoch}, Training loss: {loss_train.item():.f},'
                  f' Validation loss {loss_val.ite():.4f}')

#### 6.	Compare the performance of the neural networks to the other model you created. Which performed better? Why do you think that is?
